### Starter Section
- [x] set up imports
- [x] API - set-up, call, display
- [x] verify at least 100 records

In [1]:
# set-up basic imports
import requests
import pandas as pd
from pprint import pprint

# set-up imports for creating SQL tables
import numpy as np
from itertools import chain

#set-up imports for transferring table data to SQL (postgres)
from sqlalchemy import create_engine

In [2]:
# API KEY - though it's not needed?
api_key = 'eeec1e46-9491-4fc5-aada-f8cd1955cf46'

In [3]:
#API URL
url = 'https://api.thedogapi.com/v1/breeds'

In [4]:
# get API results
doggos = requests.get(url).json()

In [5]:
# display raw json

doggos

[{'weight': {'imperial': '6 - 13', 'metric': '3 - 6'},
  'height': {'imperial': '9 - 11.5', 'metric': '23 - 29'},
  'id': 1,
  'name': 'Affenpinscher',
  'bred_for': 'Small rodent hunting, lapdog',
  'breed_group': 'Toy',
  'life_span': '10 - 12 years',
  'temperament': 'Stubborn, Curious, Playful, Adventurous, Active, Fun-loving',
  'origin': 'Germany, France',
  'reference_image_id': 'BJa4kxc4X',
  'image': {'id': 'BJa4kxc4X',
   'width': 1600,
   'height': 1199,
   'url': 'https://cdn2.thedogapi.com/images/BJa4kxc4X.jpg'}},
 {'weight': {'imperial': '50 - 60', 'metric': '23 - 27'},
  'height': {'imperial': '25 - 27', 'metric': '64 - 69'},
  'id': 2,
  'name': 'Afghan Hound',
  'country_code': 'AG',
  'bred_for': 'Coursing and hunting',
  'breed_group': 'Hound',
  'life_span': '10 - 13 years',
  'temperament': 'Aloof, Clownish, Dignified, Independent, Happy',
  'origin': 'Afghanistan, Iran, Pakistan',
  'reference_image_id': 'hMyT4CDXR',
  'image': {'id': 'hMyT4CDXR',
   'width': 606,

In [6]:
doggos[0]['bred_for']

'Small rodent hunting, lapdog'

In [7]:
# checking that records meet 100 minimum requirement

len(doggos)

172

### Data Cleaning
- [x] get list of all attributes used and count to evaluate columns to be used
- [x] reference image id VS image id (resolved - deleted image id)

In [8]:
# create a list of all attributes for dataframe to evaluate appropriate columns for dataframe
all_attributes = []
for group in doggos:
    for line in group:
        if line not in all_attributes:
            all_attributes.append(line)

# count occurrences of attributes
att_count = {}
for attribute in all_attributes:
    count = 0
    for group in doggos:
        if attribute in group:
            count += 1
            att_count[attribute] = count
           
# all_attributes
att_count

{'weight': 172,
 'height': 172,
 'id': 172,
 'name': 172,
 'bred_for': 151,
 'breed_group': 156,
 'life_span': 172,
 'temperament': 168,
 'origin': 5,
 'reference_image_id': 172,
 'image': 172,
 'country_code': 12,
 'description': 1,
 'history': 2}

In [9]:
# check reference image id VS image id to see if either are missing
# RESULT: none were missing (counter = 0), so deleted image id 

# OLD CODE taken out of dogs list  --  img_id = puppers['image'].get('id','none')

# counter = 0
# for index, row in dogs_df.iterrows():
#     if row[9] != row[10]:
#         counter += 1
# print(counter)

### Create Main Dataframe
- [x] set-up: for loops to create list of lists
- [x] dataframe: assign columns
- [x] save dataframe to sql

In [10]:
# Set-up Dog Dataframe (list of lists)
dogs = []

for puppers in doggos:
    entry = []
    name = puppers.get('name','none')
    breedgroup = puppers.get('breed_group','none')
    temperament = puppers.get('temperament', 'none')
    weight = puppers['weight'].get('imperial','none')
    height = puppers['height'].get('imperial', 'none')
    bred_for = puppers.get('bred_for','none')
    life_span = puppers.get('life_span','none')
    origin = puppers.get('origin','none')
    country_code = puppers.get('country_code')
    ref_img_id = puppers.get('reference_image_id', 'none')
    
    entry.append(name)
    entry.append(breedgroup)
    entry.append(temperament)
    entry.append(weight)
    entry.append(height)
    entry.append(bred_for)
    entry.append(life_span)
    entry.append(origin)
    entry.append(country_code)
    entry.append(ref_img_id)
    dogs.append(entry)



# THESE are attributes I have not included
#  'description'
#  'history'
# 'image id'

In [11]:
# create dataframe
dogs_df = pd.DataFrame(dogs, columns =['Dog Breed', 
                                       'Breed Group', 
                                       'Temperament', 
                                       'Weight', 
                                       'Height', 
                                       'Bred For', 
                                       'Life Span', 
                                       'Origin', 
                                       'Country Code',
                                       'Reference Image ID'
                                      ])

dogs_df['Breed Group'] = dogs_df['Breed Group'].replace([''],'none')
dogs_df['Bred For'] = dogs_df['Bred For'].str.lower()
dogs_df = dogs_df.reset_index()
dogs_df = dogs_df.rename(columns={'index': 'Breed ID'})
dogs_df.head(20)


,Breed ID,Dog Breed,Breed Group,Temperament,Weight,Height,Bred For,Life Span,Origin,Country Code,Reference Image ID
0,0,Affenpinscher,Toy,"Stubborn, Curious, Playful, Adventurous, Activ...",6 - 13,9 - 11.5,"small rodent hunting, lapdog",10 - 12 years,"Germany, France",None,BJa4kxc4X
1,1,Afghan Hound,Hound,"Aloof, Clownish, Dignified, Independent, Happy",50 - 60,25 - 27,coursing and hunting,10 - 13 years,"Afghanistan, Iran, Pakistan",AG,hMyT4CDXR
2,2,African Hunting Dog,none,"Wild, Hardworking, Dutiful",44 - 66,30,a wild pack animal,11 years,,None,rkiByec47
3,3,Airedale Terrier,Terrier,"Outgoing, Friendly, Alert, Confident, Intellig...",40 - 65,21 - 23,"badger, otter hunting",10 - 13 years,"United Kingdom, England",None,1-7cgoZSh
4,4,Akbash Dog,Working,"Loyal, Independent, Intelligent, Brave",90 - 120,28 - 34,sheep guarding,10 - 12 years,,None,26pHT3Qk7
5,5,Akita,Working,"Docile, Alert, Responsive, Dignified, Composed...",65 - 115,24 - 28,hunting bears,10 - 14 years,none,None,BFRYBufpm
6,6,Alapaha Blue Blood Bulldog,Mixed,"Loving, Protective, Trainable, Dutiful, Respon...",55 - 90,18 - 24,guarding,12 - 13 years,none,None,33mJ-V3RX
7,7,Alaskan Husky,Mixed,"Friendly, Energetic, Loyal, Gentle, Confident",38 - 50,23 - 26,sled pulling,10 - 13 years,none,None,-HgpNnGXl
8,8,Alaskan Malamute,Working,"Friendly, Affectionate, Devoted, Loyal, Dignif...",65 - 100,23 - 25,"hauling heavy freight, sled pulling",12 - 15 years,none,None,dW5UucTIW
9,9,American Bulldog,Working,"Friendly, Assertive, Energetic, Loyal, Gentle,...",60 - 120,22 - 27,none,10 - 12 years,none,None,pk1AAdloG


### Temperaments
- [x] create list of all temperaments (resolved: 124 unique temperaments used)
- [x] count the occurrence of each temperament in dataset
- [x] sort occurrences from greatest occurrence to least
- [x] create temperament (columns) V dog breeds (rows) table

In [12]:
# create list of all temperaments (listed once)
temp_list = []
for dog in doggos:
    if 'temperament' in dog:
        temp = dog['temperament'].split(',')
        for att in temp:
            att = att.strip()
            if att not in temp_list:
                temp_list.append(att)
                
print(len(temp_list))
pprint(temp_list)

124
['Stubborn',
 'Curious',
 'Playful',
 'Adventurous',
 'Active',
 'Fun-loving',
 'Aloof',
 'Clownish',
 'Dignified',
 'Independent',
 'Happy',
 'Wild',
 'Hardworking',
 'Dutiful',
 'Outgoing',
 'Friendly',
 'Alert',
 'Confident',
 'Intelligent',
 'Courageous',
 'Loyal',
 'Brave',
 'Docile',
 'Responsive',
 'Composed',
 'Receptive',
 'Faithful',
 'Loving',
 'Protective',
 'Trainable',
 'Responsible',
 'Energetic',
 'Gentle',
 'Affectionate',
 'Devoted',
 'Assertive',
 'Dominant',
 'Strong Willed',
 'Obedient',
 'Reserved',
 'Kind',
 'Sweet-Tempered',
 'Tenacious',
 'Attentive',
 'Steady',
 'Bold',
 'Proud',
 'Reliable',
 'Fearless',
 'Lively',
 'Self-assured',
 'Cautious',
 'Eager',
 'Good-natured',
 'Spirited',
 'Companionable',
 'Even Tempered',
 'Rugged',
 'Fierce',
 'Refined',
 'Joyful',
 'Agile',
 'Amiable',
 'Excitable',
 'Determined',
 'Self-confidence',
 'Hardy',
 'Calm',
 'Good-tempered',
 'Watchful',
 'Hard-working',
 'Feisty',
 'Cheerful',
 'Sensitive',
 'Easygoing',
 'Ada

In [13]:
# creating temperament id && name table
temps_df = pd.DataFrame({'temperament_name': temp_list})
temps_df = temps_df.sort_values(by='temperament_name').reset_index(drop=True)
temps_df.index.name = 'temperament_id'
temps_df = temps_df.reset_index()

temps_df


,temperament_id,temperament_name
0,0,Active
1,1,Adaptable
2,2,Adventurous
3,3,Affectionate
4,4,Aggressive
...,...,...
119,119,Vigilant
120,120,Vocal
121,121,Watchful
122,122,Wild


In [14]:
group_list = []
for group in dogs_df['Breed Group']:
    if group not in group_list:
            group_list.append(group)

group_df = pd.DataFrame(group_list).reset_index()
group_df = group_df.rename(columns={'index': 'group_id', 0: 'group_name'})
group_df

,group_id,group_name
0,0,Toy
1,1,Hound
2,2,none
3,3,Terrier
4,4,Working
5,5,Mixed
6,6,Non-Sporting
7,7,Sporting
8,8,Herding


In [15]:
breeds = {'breed_id': dogs_df['Breed ID'], 
          'breed_name': dogs_df['Dog Breed'], 
          'min_height': 1, 
          'max_height': 1, 
          'min_weight': 1, 
          'max_weight': 1, 
          'min_life': 1, 
          'max_life': 1, 
          'group': dogs_df['Breed Group']}
breed_df = pd.DataFrame(breeds)

for group in group_df.index:
    breed_df = breed_df.replace(to_replace=group_df.iloc[group]['group_name'], value=group_df.loc[group]['group_id'])
breed_df

,breed_id,breed_name,min_height,max_height,min_weight,max_weight,min_life,max_life,group
0,0,Affenpinscher,1,1,1,1,1,1,0
1,1,Afghan Hound,1,1,1,1,1,1,1
2,2,African Hunting Dog,1,1,1,1,1,1,2
3,3,Airedale Terrier,1,1,1,1,1,1,3
4,4,Akbash Dog,1,1,1,1,1,1,4
...,...,...,...,...,...,...,...,...,...
167,167,Wire Fox Terrier,1,1,1,1,1,1,2
168,168,Wirehaired Pointing Griffon,1,1,1,1,1,1,7
169,169,Wirehaired Vizsla,1,1,1,1,1,1,7
170,170,Xoloitzcuintli,1,1,1,1,1,1,6


In [16]:
# create breed V temperament table, use https://stackoverflow.com/questions/50731229/split-cell-into-multiple-rows-in-pandas-dataframe
# use 'lookup' to replace breed with id - BOOM there's middle table (reference 62 from ETL project)

# return list from series of comma-separated strings
def chaining(s):
    return list(chain.from_iterable(s.str.split(',')))

# calculate lengths of splits
temps = dogs_df['Temperament'].str.split(',').map(len)

# create new dataframe, repeating or chaining as appropriate
dogs_temps = pd.DataFrame({'breed_id': np.repeat(dogs_df['Breed ID'], temps),
                    'temperament_name': chaining(dogs_df['Temperament'])})
dogs_temps = dogs_temps.reset_index(drop=True)

dogs_temps

,breed_id,temperament_name
0,0,Stubborn
1,0,Curious
2,0,Playful
3,0,Adventurous
4,0,Active
...,...,...
1027,171,Bold
1028,171,Independent
1029,171,Confident
1030,171,Intelligent


In [17]:
merged = dogs_temps.merge(temps_df, how='inner', on='temperament_name')
breed_temp_ids = merged[['breed_id', 'temperament_id']]

breed_temp_ids

,breed_id,temperament_id
0,0,107
1,37,107
2,46,107
3,75,107
4,138,107
...,...,...
163,125,45
164,130,70
165,143,22
166,157,123


In [18]:
# bred_for 
purpose_list = []
purpose_word_list = []
for purpose in dogs_df['Bred For']:
    purpose = purpose.split(',')
    for line in purpose:
        line = line.strip()
        line = line.lower()
        if line not in purpose_list:
            purpose_list.append(line)
        words = line.split(' ')
        for word in words:
            if word not in purpose_word_list:
                purpose_word_list.append(word)

# counting occurences of each 'bred for' category to look for similiarities between breeds
## purpose word count 

purpose_word_count = {}
purpose_count= {}

for word in purpose_word_list:
    count = 0
    for purpose in dogs_df['Bred For']:
        if word in purpose:
            count += 1
            purpose_word_count[word] = count

for phrase in purpose_list:
    count = 0
    for purpose in dogs_df['Bred For']:
        if phrase in purpose:
            count += 1
            purpose_count[phrase] = count


purpose_word_df = pd.DataFrame.from_dict(purpose_word_count, orient='index')
purpose_word_df = purpose_word_df.reset_index().rename(columns={'index': 'purpose_word', 0: 'purpose_count'}).sort_values(by='purpose_count', ascending=False)

# purpose_list
# purpose_word_list
# purpose_word_count
# purpose_count
purpose_word_df.head(20)

,purpose_word,purpose_count
31,,172
149,in,116
6,a,115
83,an,47
48,on,43
141,hunt,39
2,hunting,36
90,guard,29
23,dog,28
196,one,22


In [19]:
exclusions = ['a', 'in', '', 'an', 'on', 'and', 'one', 'none', 'to', 'the', 'or', '-', 'all', 'of', '&', 'by', 'inside', 'like', 'with']
purpose_df = purpose_word_df[~purpose_word_df['purpose_word'].isin(exclusions)]

pd.set_option("display.max_rows", None, "display.max_columns", None)
purpose_df

,purpose_word,purpose_count
141,hunt,39
2,hunting,36
90,guard,29
23,dog,28
36,herding,20
22,companion,14
34,retrieving,13
13,guarding,13
45,guardian,12
57,vermin,12


### Download tables to SQL


In [20]:
# enter password for postgres
rds_connection_string = "postgres:postgres@localhost:5432/Project_2"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [23]:
# Add data to empty tables.
group_df.to_sql(name='groups', con=engine, if_exists='append', index=False)
temps_df.to_sql(name='temperament', con=engine, if_exists='append', index=False)
breed_df.to_sql(name='breed', con=engine, if_exists='append', index=False)
breed_temp_ids.to_sql(name='breed_temperament', con=engine, if_exists='append', index=False)

### EXTRAS: Group-Bys && Counts
- [x]  group by breed group
- [ ]  group by weight
    - split string into min/max, store as min max vals (if 1 value min/max same, if 0 'no age' or NaN)
    - create list of weights based on lowest highest 
    - create new table with weights as columns V names as rows, t/f values 
    - count weights 
- [ ]  group by height
    - split string into min/max, store as min max vals (if 1 value then min/max same, if 0 then 'no age' or NaN)
    - create list of heights based on lowest + highest
    - create new table with heights as columns V names as rows, t/f values
    - count heights 
- [x] count temperament occurrences
    - create a count of all temperaments, store in dictionary
    - create a dataframe
    

In [ ]:
# Breed group
gb_breed_group = dogs_df.groupby('Breed Group')
breed_groups = pd.DataFrame(gb_breed_group['Dog Breed'].count()).reset_index().drop([0])

breed_groups

In [ ]:
# count number of times temperaments appear in dataset
temp_counts = {}

for temperament in temp_list:
    counter = 0
    for dog in doggos:
        if 'temperament' in dog:
            if temperament in dog['temperament']:
                counter += 1
                temp_counts[temperament] = counter

# create df
temp_counts_df = pd.DataFrame(temp_counts.items(), columns=['Temperament', 'Count'])


## OPTIONAL : sorted temperaments by highest occurrences
# sort temp_counts from greatest to least

temp_count_sort = pd.DataFrame.from_dict(temp_counts, 
                       orient='index', 
                       columns=['Temperament Count']).sort_values(by='Temperament Count', 
                                                                  ascending=False)
temp_count_sort.head(20)
# temp_counts_df

In [ ]:
## EXTRAS?

## create dataframe of temperaments, each dog as rows temps as columns
# list of lists, dog name as first list value, t/f for each temperament
all_dogs = []
for dog in doggos:
    dog_list = [dog['name']]
    if 'temperament' in dog:
        for temperament in temp_list:
            if temperament in dog['temperament']:
                dog_list.append('True')
            else:
                dog_list.append('False')
    all_dogs.append(dog_list)

# create table with list of lists, add 'Name' to temp_list to accommodate first list value
column_names = ['Name'] + temp_list
temp_dogs_table = pd.DataFrame(all_dogs, columns= column_names)
temp_dogs_table